# LDA using Gibb's sampling

Op basis van uitleg in http://u.cs.biu.ac.il/~89-680/darling-lda.pdf

Using Gensim functionality

Dus we mogen ervan uitgaan dat we een dictionary hebben en bow vectors voor de documenten.
Het een en ander aan counts komt dan uit gensim. Maar hoe?

Wat hebben we nodig?

* The number of words assigned to topic k in document d: $$n_{d;k}$$
* The number of times word w is assigned to topic k: $$n_{k;w}$$
* The total number of times any word is assigned to topic k: $$n_k$$
* Array z which will contain the current topic assignment for each of the N words in the corpus.


In [1]:
# Generate documents
import numpy as np

length = 100
num_topics = 3
vocabulary = np.array(['zon', 'ijs', 'strand', 'vanille', 'chocola', 'broccoli', 'wortel'])

real_theta = np.array([[0.5, 0.0, 0.5],
                       [0.3, 0.3, 0.4],
                       [0.1, 0.8, 0.1],
                       [0.7, 0.3, 0.0],
                       [0.4, 0.4, 0.2]])
real_phi = np.array([[0.4, 0.2, 0.4, 0.0, 0.0, 0.0, 0.0],
                     [0.0, 0.3, 0.0, 0.35, 0.35, 0.0, 0.0],
                     [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5]])

In [2]:
# generate the corpus
corpus = []

for m, tm in enumerate(real_theta):
    doc = []
    for i in range(length):
        # sample topic
        topic = np.random.multinomial(1, tm)
        topic_index = np.where(topic==1)[0][0]
        word = np.random.multinomial(1, real_phi[topic_index])
        doc.append(vocabulary[np.where(word==1)[0][0]])
    corpus.append(doc)
    
for i, doc in enumerate(corpus):
    print 'document', i+1, doc



document 1 ['wortel', 'broccoli', 'strand', 'broccoli', 'strand', 'broccoli', 'zon', 'broccoli', 'ijs', 'broccoli', 'broccoli', 'strand', 'zon', 'wortel', 'zon', 'wortel', 'wortel', 'zon', 'ijs', 'wortel', 'broccoli', 'ijs', 'ijs', 'wortel', 'strand', 'wortel', 'strand', 'strand', 'broccoli', 'zon', 'broccoli', 'broccoli', 'wortel', 'strand', 'wortel', 'zon', 'strand', 'zon', 'zon', 'ijs', 'zon', 'zon', 'ijs', 'broccoli', 'wortel', 'zon', 'wortel', 'wortel', 'wortel', 'broccoli', 'ijs', 'broccoli', 'zon', 'strand', 'broccoli', 'broccoli', 'broccoli', 'ijs', 'broccoli', 'ijs', 'wortel', 'wortel', 'zon', 'zon', 'wortel', 'strand', 'strand', 'wortel', 'zon', 'wortel', 'strand', 'wortel', 'broccoli', 'wortel', 'ijs', 'broccoli', 'wortel', 'wortel', 'broccoli', 'strand', 'zon', 'wortel', 'wortel', 'zon', 'strand', 'wortel', 'zon', 'strand', 'wortel', 'wortel', 'ijs', 'broccoli', 'broccoli', 'zon', 'wortel', 'zon', 'ijs', 'broccoli', 'strand', 'zon']
document 2 ['wortel', 'vanille', 'ijs', '

In [31]:
from gensim import corpora

d = corpora.Dictionary(corpus)

In [9]:
# initialize z
import random
from collections import Counter

z = {}
ndk = np.zeros((len(corpus), len(real_phi)), dtype=np.int)
nkw = np.zeros((len(real_phi), len(vocabulary)), dtype=np.int)
nk = Counter()

for d, doc in enumerate(corpus):
    for i, word in enumerate(doc):
        t = random.randint(0,2)
        z[(d, i)] = t
        ndk[d][t] += 1
        word_index = np.where(vocabulary==word)[0][0]
        nkw[t][word_index] += 1
        nk[t] += 1

print z
print ndk
print np.sum(ndk)
print nkw
print np.sum(nkw)
print nk

{(0, 86): 1, (3, 35): 0, (4, 36): 2, (2, 84): 0, (0, 76): 1, (4, 66): 1, (1, 64): 2, (2, 78): 0, (0, 98): 0, (3, 86): 0, (0, 17): 1, (1, 28): 2, (2, 27): 1, (0, 55): 1, (3, 2): 2, (1, 54): 0, (4, 5): 1, (2, 53): 2, (0, 45): 0, (3, 40): 2, (1, 40): 2, (4, 35): 0, (2, 47): 1, (0, 67): 2, (4, 73): 1, (1, 89): 1, (2, 73): 2, (3, 95): 0, (4, 87): 2, (2, 99): 2, (1, 21): 1, (2, 18): 2, (0, 14): 0, (3, 11): 1, (1, 15): 0, (4, 12): 1, (2, 12): 0, (0, 36): 2, (3, 17): 2, (1, 33): 2, (4, 42): 2, (2, 38): 2, (0, 90): 0, (3, 55): 0, (4, 48): 2, (1, 82): 0, (2, 64): 2, (3, 68): 1, (4, 94): 0, (0, 5): 1, (1, 0): 2, (4, 11): 0, (2, 7): 0, (0, 59): 2, (3, 22): 2, (1, 58): 0, (4, 17): 1, (2, 33): 2, (0, 81): 2, (3, 60): 0, (4, 63): 0, (2, 91): 2, (3, 77): 0, (4, 69): 1, (1, 77): 2, (3, 83): 2, (4, 99): 2, (0, 28): 1, (1, 25): 2, (2, 30): 2, (0, 50): 0, (3, 31): 2, (1, 51): 0, (4, 24): 2, (2, 56): 2, (0, 40): 1, (3, 37): 0, (4, 38): 1, (2, 82): 0, (0, 78): 2, (4, 76): 1, (1, 70): 2, (2, 76): 0, (3, 88):

In [10]:
# iterate

def p_z(alpha, beta, num_topics, num_words, d, topic, word_index):
    #print alpha, beta, num_topics, num_words, d, topic, word_index
    #print ndk[d][topic], nkw[topic][word_index], nk[topic]
    return (ndk[d][topic]*alpha)*(nkw[topic][word_index]+beta)/(nk[topic]+beta*num_words)

def normalize(p):
    minimum = np.min(p)
    maximum = np.max(p)
    
    if minimum < 0:
        return normalize((p - minimum)/(maximum - minimum))
    return p/sum(p)

num_iter = 100

alpha = 0.02
beta = 0.02

theta = np.zeros((num_iter,  len(corpus), num_topics))
phi = np.zeros((num_iter, num_topics, len(vocabulary)))

for t in range(num_iter):
    for d, doc in enumerate(corpus):
        for i, w in enumerate(doc):
            word = (d, i)
            topic = z[word]
                        
            word_index = np.where(vocabulary==w)[0][0]
                
            #if ndk[d][topic] > 0 and nkw[topic][word_index] > 0 and nk[topic] > 0:
            if True:
                ndk[d][topic] -= 1
                nkw[topic][word_index] -= 1
                nk[topic] -= 1
            
                p = [p_z(alpha, beta, len(real_phi), len(vocabulary), d, j, word_index) for j in range(len(real_phi))]
                #print p
                # normalize
                p = normalize(p)
                #print p, sum(p)
            
                to = np.random.multinomial(1, p)
                topic = np.where(to==1)[0][0]
                
                z[word] = topic
            
                ndk[d][topic] += 1
                nkw[topic][word_index] += 1
                nk[topic] += 1
    # calculate theta and phi
    theta[t] = (ndk+float(alpha))/(np.sum(ndk, axis=1, keepdims=True)+num_topics*alpha)
    phi[t] = (nkw+float(beta))/(np.sum(nkw, axis=1, keepdims=True)+len(vocabulary)*beta)
            #print 'new topic', topic
print ndk
print nkw
print nk

[[  0  37  63]
 [ 53   0  47]
 [100   0   0]
 [ 27  73   0]
 [ 28  48  24]]
[[21 54  2 65 58  2  6]
 [73 44 30  0 11  0  0]
 [ 0  0 26  0  0 47 61]]
Counter({0: 208, 1: 158, 2: 134})


In [11]:
print 'theta'
print 'found'
print np.mean(theta, axis=0)
print 'real'
print real_theta
print
print 'phi'
print 'found'
print np.mean(phi, axis=0)
print 'real'
print real_phi
print
print 'topics found:'
indexes = np.mean(phi, axis=0) > 0.01
for index in indexes:
    print vocabulary[index]
print
print 'topics real'
indexes = real_phi > 0.01
for index in indexes:
    print vocabulary[index]

theta
found
[[ 0.00239856  0.33569858  0.66190286]
 [ 0.45712572  0.0064961   0.53637817]
 [ 0.9895063   0.00829502  0.00219868]
 [ 0.33439936  0.66060364  0.004997  ]
 [ 0.3971617   0.35978413  0.24305417]]
real
[[ 0.5  0.   0.5]
 [ 0.3  0.3  0.4]
 [ 0.1  0.8  0.1]
 [ 0.7  0.3  0. ]
 [ 0.4  0.4  0.2]]

phi
found
[[ 0.0921228   0.24737625  0.01099439  0.29633911  0.30185129  0.01459099
   0.03672517]
 [ 0.43187753  0.29995454  0.24493072  0.00216886  0.00874174  0.00634798
   0.00597863]
 [ 0.09803914  0.01669008  0.15297795  0.00137759  0.01012465  0.31450553
   0.40628508]]
real
[[ 0.4   0.2   0.4   0.    0.    0.    0.  ]
 [ 0.    0.3   0.    0.35  0.35  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.5   0.5 ]]

topics found:
['zon' 'ijs' 'strand' 'vanille' 'chocola' 'broccoli' 'wortel']
['zon' 'ijs' 'strand']
['zon' 'ijs' 'strand' 'chocola' 'broccoli' 'wortel']

topics real
['zon' 'ijs' 'strand']
['ijs' 'vanille' 'chocola']
['broccoli' 'wortel']
